In [ ]:
import pandas as pd
import numpy as np

## 루트 디렉토리 설정 

In [ ]:
# 작업 디렉토리 설정
path = r'D:\2020 구매기획팀\3. 정기업무\7. 포상 대상자 분석\파일 새로만들기\템플릿 업그레이드'

In [ ]:
# 실적 데이터
file_name = r'구매실적.xlsx'

df = pd.read_excel(path + '\\' + file_name , encoding='cp949')
df

In [ ]:
df['절감률'] = df['절감액'] / (df['구매금액'] + df['절감액'])
df.replace(np.nan, 0)
df.head(1)

## 구매금액, 절감액, 절감률 

In [ ]:
df_saving_rate = df.groupby(['담당자'])[['구매금액', '절감액', '도전목표']].sum()
df_saving_rate = pd.DataFrame(df_saving_rate)
df_saving_rate['절감률'] = df_saving_rate['절감액'] / (df_saving_rate['구매금액'] + df_saving_rate['절감액'])
df_saving_rate

## 목표달성률  

### 원료 

+ 목표 > 0: 

<blockquote>목표 달성률 = 1+ {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

+ 목표 < 0: 

<blockquote>목표 달성률 = {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

### 설비

+ 설비/외자: 절감률 / 7%
+ 순환: 제외 

### 인테리어

+ 인테리어: 절감률 / 4.5%

In [ ]:
df.loc[df['도전목표'] < 0, '목표달성률'] = (df['절감액'] - df['도전목표']) / abs(df['도전목표'])
df.loc[df['도전목표'] == 0, '목표달성률'] = df['목표 달성률'] = 0
df.loc[df['도전목표'] > 0, '목표달성률'] = 1 + \
    ((df['절감액'] - df['도전목표']) / abs(df['도전목표']))
df.loc[df['구분'] == '설비', '목표달성률'] = df['절감률'] / 0.07
df.loc[df['구분'] == '외자', '목표달성률'] = df['절감률'] / 0.07 #인수인계는 5.5%였으나, 김재욱 차장님이 7% 라고 하심
df.loc[df['구분'] == '인테리어', '목표달성률'] = df['절감률'] / 0.045

In [ ]:
df['index'] = df.loc[(df['구분']!='순환')].groupby('담당자')['구매금액'].rank(method='first', ascending=False)
df_achieved_rate = df[['팀', '담당자', '구분','구매금액','절감액','도전목표','절감률','목표달성률', 'index']]
df_achieved_rate_filtered = df_achieved_rate.loc[(df_achieved_rate['index'] < 2)]
df_achieved_rate = df_achieved_rate_filtered[['팀', '담당자','구분', '목표달성률', 'index']]
df_achieved_rate

In [ ]:
df_saving = pd.merge(df_saving_rate, df_achieved_rate, how='left', on='담당자')
df_saving[['팀', '담당자','구분', '구매금액','도전목표', '절감액', '절감률','목표달성률']]
df_saving

## 신규아이디어 제안 성과 분석

### 신규절감제안 절감률 
+ 월 구매금액 대비 신규 절감 아이디어 제안 금액의 절감률
+ 신규아이디어 제안서 상정주차 기준으로 포상 기준월 담당자별 절감액 산출

In [ ]:
# 신규아이디어 절감액 데이터 import
file_name2 = r'아이디어절감액.xlsx'
df2 = pd.read_excel(path + '\\' + file_name2 , encoding='cp949')
df2.head(1)

In [ ]:
df_sum = df.groupby('담당자')['구매금액'].sum()
df_sum = pd.DataFrame(df_sum)
df_sum.head(1)

In [ ]:
df_merge = pd.merge(df_sum, df2, how='inner', on='담당자')
df_merge['제안절감률'] = df_merge['아이디어절감액'] / df_merge['구매금액']
df_idea_saving = df_merge[['담당자','아이디어절감액', '제안절감률']]
df_idea_saving

### 신규절감제안 달성률
+ 신규 아이디어 제안의 당월 목표 대비 절감 달성률
+ 원가절감 Tracking 목표 및 절감액 

<blockquote>신규절감제안 달성률 = 당월 제안절감액 / 당월 제안목표</blockquote>

In [ ]:
# 원가절감 Tracking 데이터 import
file_name3 = r'원가절감Tracking.xlsx'
df3 = pd.read_excel(path + '\\' + file_name3 , encoding='cp949')
df3.head(1)

In [ ]:
df3['제안달성률'] = df3['제안절감액'] / df3['제안목표']
df_idea_achieved = df3
df_idea_achieved.head(5)

## 상품개발 성과 분석

### 상품개발 목표 대비 달성률 & 3개월 매출 평균  

+ 상품개발 달성률: 신규 상품개발의 당월 목표 대비 매출 달성률
<blockquote> 상품개발 달성률 = 당월 매출/ 상품개발목표 </blockquote>

+ 상품개발 매출액 평균: 최근 3개월 매출 평균금액 (단, 1월, 2월은 해당 월 까지의 누계 평균 반영)

In [ ]:
# 원가절감 Tracking 데이터 import
file_name4 = r'상품개발실적.xlsx'
df4 = pd.read_excel(path + '\\' + file_name4 , encoding='cp949')
df4.head(1)

In [ ]:
df4['매출목표달성률'] = df4['당월매출'] / df4['상품개발목표']
#df4_revenue = df4_revenue_pre.
df_revenue = df4.replace(np.inf, 0)
df_revenue.head(1)

## Data Merge

In [ ]:
df_merged1 = pd.merge(df_saving, df_idea_saving, how='left', on='담당자')
df_merged2 = pd.merge(df_merged1, df_idea_achieved, how='left', on='담당자')
df_merged3 = pd.merge(df_merged2, df_revenue, how='left', on='담당자')
df_merged = df_merged3.replace(np.nan, 0)
df_merged.head(1)

## Create scores

In [ ]:
df_merged['Score_절감목표달성'] = (
    np.select(
        condlist=[df_merged['목표달성률'] < -0.5, df_merged['목표달성률'] < 0.0, 
                  df_merged['목표달성률'] < 0.5, df_merged['목표달성률'] < 0.8, 
                  df_merged['목표달성률'] < 1.0, df_merged['목표달성률'] < 1.5,
                  df_merged['목표달성률'] < 2.0, df_merged['목표달성률'] >= 2.0],
        choicelist=[30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

df_merged['Score_절감액'] = (
    np.select(
        condlist=[df_merged['절감액'] < -100000000, df_merged['절감액'] < 0, 
                  df_merged['절감액'] < 50000000, df_merged['절감액'] < 100000000, 
                  df_merged['절감액'] < 150000000, df_merged['절감액'] < 200000000,
                  df_merged['절감액'] < 300000000, df_merged['절감액'] >= 300000000],
        choicelist=[30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

df_merged['Score_구매절감률'] = (
    np.select(
        condlist=[df_merged['절감률'] <= 0.0, df_merged['절감률'] < 0.002, 
                  df_merged['절감률'] < 0.005, df_merged['절감률'] < 0.01, 
                  df_merged['절감률'] < 0.015, df_merged['절감률'] < 0.02,
                  df_merged['절감률'] < 0.025, df_merged['절감률'] < 0.03, 
                  df_merged['절감률'] >= 0.03],
        choicelist=[0, 30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

# 제안절감률 점수 0점은 미제안 시 부여하도록 되어 있으나, 현재 수식은 제안절감률이 0.0 이하인 경우로 설정되어 있음
df_merged['Score_제안절감률'] = (
    np.select(
        condlist=[df_merged['제안절감률'] <= 0.0, df_merged['제안절감률'] < 0.002, 
                  df_merged['제안절감률'] < 0.005, df_merged['제안절감률'] < 0.01, 
                  df_merged['제안절감률'] < 0.015, df_merged['제안절감률'] < 0.02,
                  df_merged['제안절감률'] < 0.025, df_merged['제안절감률'] < 0.03, 
                  df_merged['제안절감률'] >= 0.03],
        choicelist=[0, 30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

# 제안달성률 점수 0점은 미제안 시 부여하도록 되어 있으나, 현재 수식은 제안달성률이 0.0 이하인 경우로 설정되어 있음
df_merged['Score_제안달성률'] = (
    np.select(
        condlist=[df_merged['제안달성률'] <= 0.0, df_merged['제안달성률'] < 0.6, 
                  df_merged['제안달성률'] < 0.7, df_merged['제안달성률'] < 0.8, 
                  df_merged['제안달성률'] < 0.9, df_merged['제안달성률'] < 1.0,
                  df_merged['제안달성률'] < 1.5, df_merged['제안달성률'] < 2.0, 
                  df_merged['제안달성률'] >= 2.0],
        choicelist=[0, 30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

df_merged['Score_매출목표달성률'] = (
    np.select(
        condlist=[df_merged['매출목표달성률'] <= 0.0, df_merged['매출목표달성률'] < 0.6, 
                  df_merged['매출목표달성률'] < 0.7, df_merged['매출목표달성률'] < 0.8, 
                  df_merged['매출목표달성률'] < 0.9, df_merged['매출목표달성률'] < 1.0,
                  df_merged['매출목표달성률'] < 1.2, df_merged['매출목표달성률'] < 1.5, 
                  df_merged['매출목표달성률'] >= 1.5],
        choicelist=[0, 30, 40, 50, 60, 70, 80, 90, 100],
        default=0))

df_merged['Score_매출액'] = (
    np.select(
        condlist=[df_merged['3개월매출평균'] == 0, df_merged['3개월매출평균'] < 5000000, 
                  df_merged['3개월매출평균'] < 10000000, df_merged['3개월매출평균'] < 15000000, 
                  df_merged['3개월매출평균'] < 20000000, df_merged['3개월매출평균'] < 25000000,
                  df_merged['3개월매출평균'] >= 25000000],
        choicelist=[0, 10, 20, 40, 60, 80, 100],
        default=0))

df_score = df_merged[['팀', '담당자', '구매금액', '도전목표', '절감액', '상품개발목표', '구분', 'Score_절감목표달성', 'Score_절감액', 'Score_구매절감률', 'Score_제안절감률', 'Score_제안달성률', 'Score_매출목표달성률', 'Score_매출액']]
df_score

## Create weighted scores

In [ ]:
# WS means 'Weighted Score'

df_score2 = df_score.copy()

df_score2['WS_절감목표달성'] = df_score2['Score_절감목표달성'] * 0.3


df_score2['WS_절감액'] = (
    np.select(
        condlist=[df_score['구분'] == '설비', df_score['구분']
                  == '외자', df_score['구분'] == '인테리어'],
        choicelist=[df_score['Score_절감액'] * 0.3,
                    df_score['Score_절감액'] * 0.3, 0],
        default=df_score['Score_절감액'] * 0.15))

df_score2['WS_구매절감률'] = (
    np.select(
        condlist=[df_score['구분'] == '설비', df_score['구분']
                  == '외자', df_score['구분'] == '인테리어'],
        choicelist=[0, 0, df_score['Score_구매절감률'] * 0.3],
        default=df_score['Score_구매절감률'] * 0.15))


# 제안 점수는 상품개발 목표 유무에 따라 기준을 달리 적용
# 상품개발 목표가 없으면 가중치 20%, 상품개발 목표가 있으면 가중치 10%
df_score2['WS_제안절감률'] = (
    np.select(
        condlist=[df_score['상품개발목표'] == 0, df_score['상품개발목표'] != 0],
        choicelist=[df_score['Score_제안절감률'] *
                    0.2, df_score['Score_제안절감률'] * 0.1],
        default=df_score['Score_제안절감률'] * 0.1))

df_score2['WS_제안달성률'] = (
    np.select(
        condlist=[df_score['상품개발목표'] == 0, df_score['상품개발목표'] != 0],
        choicelist=[df_score['Score_제안달성률'] *
                    0.2, df_score['Score_제안달성률'] * 0.1],
        default=df_score['Score_제안달성률'] * 0.1))

df_score2['WS_매출목표달성률'] = (
    np.select(
        condlist=[df_score['상품개발목표'] == 0, df_score['상품개발목표'] != 0],
        choicelist=[0, df_score['Score_매출목표달성률'] * 0.1],
        default=df_score['Score_매출목표달성률'] * 0.1))

df_score2['WS_매출액'] = (
    np.select(
        condlist=[df_score['상품개발목표'] == 0, df_score['상품개발목표'] != 0],
        choicelist=[0, df_score['Score_매출액'] * 0.1],
        default=df_score['Score_매출액'] * 0.1))

df_score2['WS_합계'] = df_score2['WS_절감목표달성'] + df_score2['WS_절감액'] + df_score2['WS_구매절감률'] + \
    df_score2['WS_제안절감률'] + df_score2['WS_제안달성률'] + \
    df_score2['WS_매출목표달성률'] + df_score2['WS_매출액']

df_result = df_score2.sort_values(by='WS_합계', ascending=False).reset_index()
df_result.to_clipboard()